# Running [2,0,2] Experiments

In [ ]:
from qiskit_qec.circuits.repetition_code import ArcCircuit
from qiskit_ibm_provider import IBMProvider
from qiskit_aer import AerSimulator, noise

import pickle

provider = IBMProvider(instance='ibm-q-internal/deployed/default')

print('''
qiskit-terra	0.24.1
qiskit-aer	0.12.1
qiskit-ibmq-provider	0.20.2
qiskit	0.43.2
System information
Python version	3.10.6
Python compiler	Clang 12.0.0
Python build	main, Oct 24 2022 11:04:34
''')

The following cell sets up backends.

* `backend`: Backend for a real device. Used if `simulate == False`.
* `sim_backend`: The Aer simulator. Used if `simulate == True`.
* `p`: Probability of noise (used for all arguments in the function below).
* `get_noise_model(p_1,p_2,p_m)`: Creates a noise model to use with the Aer simulator, using the given values for error probabilities.

In [ ]:
simulate = False

if not simulate:
    backend = provider.get_backend('ibm_sherbrooke')
else:
    backend = AerSimulator()
    p_all = 0.01

def get_noise_model(p_1,p_2,p_m):
    e_1 = noise.depolarizing_error(p_1, 1)
    e_2 = noise.depolarizing_error(p_2, 2)
    e_m = noise.pauli_error([('X',p_m),('I',1-p_m)])
    noise_model = noise.NoiseModel()
    noise_model.add_all_qubit_quantum_error(e_1, ['h', 'x', 'rz', 'sx'])
    noise_model.add_all_qubit_quantum_error(e_2, ['cx'])
    noise_model.add_all_qubit_quantum_error(e_m, ['measure'])
    return noise_model

Now we can choose the links on which to define our code and the schedule to use.

In [ ]:
if simulate:
    processor = 'simulator'
else:
    processor = backend.configuration().processor_type['family']

if processor == 'Falcon': # for 27 qubit devices
    links = [(0,1,4),(4,7,10),(10,12,15)]
    schedule = [[(0,1),(4,7),(10,12)],[(4,1),(10,7),(15,12)]]
else: # for simulations or 'Eagle' devices
    links = [(2*j,2*j+1,2*j+2) for j in range(3)]
    schedule = None


And then we can set up the code.

We use the following parameters:
* `T=10` is (1 more than the default `code.rounds_per_202`).
* `run_202=True` because we want to run the `[[2,0,2]]`s;
* `conditional_reset=True` because the `[[2,0,2]]`s require conditional operations, so we might as well use them for the resets too;
* `barriers=True` because, at the very least, they make circuit diagrams easier to understand.

In [ ]:
code = ArcCircuit(
        links,
        10,
        schedule=schedule,
        run_202=True,
        conditional_reset=True,
        barriers=True,
    )

If we are simulating, let's see what the results would look like without noise.

In [ ]:
if simulate:
    backend.run(code.circuit[code.base]).result().get_counts()

Then we can run this on the actual intended backend. Note that we need to use `dynamic=True`, because the circuit uses conditional gates.

In [ ]:
if simulate:
    qc = [code.circuit[basis] for basis in [code.basis, code.basis[::-1]]]
    job = backend.run(qc, shots=4000, noise_model=get_noise_model(p_all,p_all,p_all))
else:
    qc = [code.transpile(backend, echo=('X','X'), echo_num=(2,2))[basis] for basis in [code.basis, code.basis[::-1]]]
    job = backend.run(qc, dynamic=True)
    print(job.job_id())

if not simulate:
    job_id = job.job_id()
    print(job_id)

Here are some jobs that have already been run.

*Without dd (25th May)*

* `'ibm_sherbrooke'` 'chnn2hanajhpa63smvd0'

*With `echo=('X','X'), echo_num=(2,2)` (5th June)*

* `'ibm_sherbrooke'` 'chur07ii3durlgru8vsg'

*With `echo=('X','X'), echo_num=(2,0)` (5th June)*

* `'ibm_sherbrooke'` 'chuss30recnk2p5n0q8g'

In [ ]:
job_id = 'chur07ii3durlgru8vsg'

if not simulate:
    try:
        with open('result/'+job_id+'.p', 'rb') as file:
            result = pickle.load(file)
    except:
        result = provider.retrieve_job(job_id).result()
        with open('result/'+job_id+'.p', 'wb') as file:
            pickle.dump(result, file)
else:
    result = job.result()
counts = result.get_counts()

Real devices mean noise (or, if using a fake backend, simulations of noise mean that there is noise). The counts dictionaries will therefore have lots of the results. Let's just look at the most common outputs.

In [ ]:
shot_limit = 50
print('Results for outputs with more than '+str(shot_limit)+' shots.')

bases = {code.basis, code.basis[::-1]}
for j, basis in enumerate(bases):
    print('\nFor basis='+basis)
    for string, shots in counts[j].items():
        if shots>shot_limit:
            print(str(shots)+' shots for output '+string)

To make sense of the results, we need to look at the decoding graph.

In [ ]:
from qiskit_qec.decoders import DecodingGraph
from qiskit_qec.decoders.decoding_graph import make_syndrome_graph_from_aer

from rustworkx.visualization import mpl_draw

The automatic graph generator isn't great for ARCs, and even worse when there are `[[2,0,2]]`s, so we use `brute=True` to get a brute force generated one.

In [ ]:
graph, hyperedges, hyperedge_errors, error_circuit = make_syndrome_graph_from_aer(code)
dg = DecodingGraph(code, brute=True, graph=graph)
mpl_draw(dg.graph)

To make some sense of this, let's look at the nodes of the graph and their indices.

In [ ]:
boundary_nodes = []
for n, node in enumerate(dg.graph.nodes()):
    print('The index is',n,'for the node',node)
    if node.is_boundary:
        boundary_nodes.append(n)

Using `'element'` and `'time'` as coordinates, we can reposition the nodes. We can also color them based on `'conjugate'`.

In [ ]:
pos = []
node_color = []
for n, node in enumerate(dg.graph.nodes()):
    if not node.is_boundary:
        pos.append( (node.index, node.time) )
    else:
        pos.append((code.d*(1-node.index)-1, code.T/2))
    if node.properties['conjugate']:
        node_color.append('lightgreen')
    else:
        node_color.append('green')

# some repositioning for the main case of interest
if code.T==10 and code.rounds_per_202==9:
    pos[0] = (pos[0][0]+1/3, pos[0][1])
    pos[1] = (pos[1][0]+1/3, pos[1][1]-1/3)
    pos[2] = (pos[2][0], pos[2][1]-6)
    pos[3] = (pos[3][0]-2/3, pos[3][1]+2.25)
    pos[4] = (pos[4][0]-1/4, pos[4][1])
    # the 202 column
    for n in [8, 6, 10, 11, 12, 13, 14]:
        pos[n] = (pos[n][0], pos[n][1]-3.25)
    # of which standard
    for n in [6, 10, 12, 14]:
        pos[n] = (pos[n][0] - 0.6, pos[n][1])
    # or conjugate
    for n in [8, 11, 13]:
        pos[n] = (pos[n][0] + 0.6, pos[n][1])
    # top
    for n in [17, 16, 15]:
        pos[n] = (pos[n][0], pos[n][1] + 1)

mpl_draw(dg.graph, pos=pos, node_color=node_color, with_labels=True)

Edges of this graph correspond to pairs of nodes that are affected by the same error. In some cases, the same error triggers multiple edges. You can see what is triggered by a set of single qubit errors below by putting in a valid edge. Here the two nodes in each edge are referred to by their indices.

In [ ]:
edge = ()

for j, hyperedge in enumerate(hyperedges):
    if edge in hyperedge:
        print(list(hyperedge.keys()), 'caused by', hyperedge_errors[j])

We can also look at these single qubit errors with their circuit diagram. Just copy the tuple specifying the error into the cell below.

We can analyze results from experiments to determine the probability of error for each edge.

In [ ]:
shots = sum(counts[0].values()) + sum(counts[1].values())
def stnd_err (p, shots):
    return (p*(1-p)/shots)**0.5

raw_error_probs = []
for j in range(2):
    raw_error_probs.append(dg.get_error_probs(counts[j], method='naive'))

# average over the two basis choices
error_probs = {}
for edge in raw_error_probs[0]:
    error_probs[edge] = raw_error_probs[0][edge]/2
for edge in raw_error_probs[1]:
    error_probs[edge] += raw_error_probs[1][edge]/2

max_prob = max(max(error_probs.values()),1e-10)
max_edge = list(error_probs.keys())[list(error_probs.values()).index(max_prob)]
print('Max probability is', max_prob, 'on edge', max_edge)

edge_color = []
for edge in dg.graph.edge_list():
    if edge not in error_probs:
        # these are associated with the boundary, and appear as loops in error_probs
        # they need to be converted to this standard form
        bulk_n = list(set(edge).difference(set(boundary_nodes)))[0]
        edge = (bulk_n, bulk_n)
    prob  = max(0,error_probs.get(edge))
    r = prob/max_prob
    edge_color.append((1-r,1-r,1-r))

if simulate:
    print('### p =',p_all)
else:
    print('###',backend.name, job_id)

print('\nProb of errors including feedforward failure')
for p in [error_probs[9,5], error_probs[5,7]]:
    print('*',round(p,3),'\pm',round(stnd_err(p,shots),3))
print('\nProb of conjugate error during [[2,0,2]]')
for p in [error_probs[11,11]]:
    print('*',round(p,3),'\pm',round(stnd_err(p,shots),3))
print('\nProb of corresponding standard errors during [[2,0,2]]')
for p in [error_probs[10,5], error_probs[12,5]]:
    print('*',round(p,3),'\pm',round(stnd_err(p,shots),3))

mpl_draw(dg.graph, pos=pos, node_color=node_color, with_labels=True, edge_color=edge_color, width=4)

Here's it all together in a loop to get simulated data.

In [ ]:
backend = AerSimulator()
sim_results = {}
for p_all in [0.025,0.035,0.045]:

    sim_results[p_all] = {}

    qc = [code.circuit[basis] for basis in [code.basis, code.basis[::-1]]]
    job = backend.run(qc, shots=10000, noise_model=get_noise_model(p_all,p_all,p_all))

    counts = job.result().get_counts()

    shots = sum(counts[0].values()) + sum(counts[1].values())
    sim_results[p_all]['shots'] = shots

    raw_error_probs = []
    for j in range(2):
        raw_error_probs.append(dg.get_error_probs(counts[j], method='naive'))

    # average over the two basis choices
    error_probs = {}
    for edge in raw_error_probs[0]:
        error_probs[edge] = raw_error_probs[0][edge]/2
    for edge in raw_error_probs[1]:
        error_probs[edge] += raw_error_probs[1][edge]/2

    max_prob = max(max(error_probs.values()),1e-10)
    max_edge = list(error_probs.keys())[list(error_probs.values()).index(max_prob)]

    sim_results[p_all]['max_prob'] = max_prob
    sim_results[p_all]['max_edge'] = max_edge
    sim_results[p_all]['ffwd'] = [error_probs[8,5], error_probs[5,7]]
    sim_results[p_all]['conjugate'] = [error_probs[11,11]]
    sim_results[p_all]['standard'] = [error_probs[10,5], error_probs[12,5]]

print(sim_results)

Here's some results we made earlier.

In [ ]:
sim_results = {0.005: {'max_prob': 0.04385, 'max_edge': (11, 11), 'ffwd': [0.02617954640776487, 0.025669610577738262], 'conjugate': [0.04385], 'standard': [0.03064930131500469, 0.0334915957900992], 'shots': 20000}, 0.01: {'max_prob': 0.09214359277430423, 'max_edge': (14, 5), 'ffwd': [0.06275398063366017, 0.058558326587641964], 'conjugate': [0.0844], 'standard': [0.06170214192112663, 0.07113803108176885], 'shots': 20000}, 0.02: {'max_prob': 0.17765479395530792, 'max_edge': (14, 5), 'ffwd': [0.12955341957763367, 0.1306555751744694], 'conjugate': [0.16160000000000002], 'standard': [0.14222081987842977, 0.14750643626065957], 'shots': 20000}, 0.03: {'max_prob': 0.24759435237433655, 'max_edge': (14, 5), 'ffwd': [0.20446450958325801, 0.19526345922622518], 'conjugate': [0.21744999999999998], 'standard': [0.1952473852769215, 0.19961036039195698], 'shots': 20000}, 0.04: {'max_prob': 0.3141572994965865, 'max_edge': (14, 5), 'ffwd': [0.2494773826003468, 0.2524715786135514], 'conjugate': [0.25384999999999996], 'standard': [0.26796004148399344, 0.25921296418949646], 'shots': 20000}, 0.05: {'max_prob': 0.35054644484435427, 'max_edge': (14, 5), 'ffwd': [0.2922905250792368, 0.29007927212258994], 'conjugate': [0.3156], 'standard': [0.29906960950496175, 0.29253184092878876], 'shots': 20000}, 0.06: {'max_prob': 0.3870570384072558, 'max_edge': (14, 5), 'ffwd': [0.3272800355932547, 0.3316617283292591], 'conjugate': [0.3399], 'standard': [0.3428667581291205, 0.33632304450769274], 'shots': 20000}, 0.07: {'max_prob': 0.40643702174480306, 'max_edge': (14, 5), 'ffwd': [0.35748184448769676, 0.3572919107027467], 'conjugate': [0.36040000000000005], 'standard': [0.3624054570119817, 0.3658368611312566], 'shots': 20000}, 0.08: {'max_prob': 0.4285738455101742, 'max_edge': (14, 5), 'ffwd': [0.3730612064983211, 0.3763210221960224], 'conjugate': [0.3983], 'standard': [0.38407127220462667, 0.38996053117990936], 'shots': 20000}, 0.09: {'max_prob': 0.4437319340141993, 'max_edge': (14, 5), 'ffwd': [0.39278051261845337, 0.39324911138952034], 'conjugate': [0.41355], 'standard': [0.40730187625205727, 0.4096979050714454], 'shots': 20000}, 0.1: {'max_prob': 0.45716763983073594, 'max_edge': (14, 5), 'ffwd': [0.41389449902922393, 0.4282831082443056], 'conjugate': [0.43565], 'standard': [0.42364918767849363, 0.42417957927900535], 'shots': 20000}, 0.015: {'max_prob': 0.13596671587342823, 'max_edge': (14, 5), 'ffwd': [0.09380215673425105, 0.09603667571207253], 'conjugate': [0.11935000000000001], 'standard': [0.1014739208663521, 0.10999189867875794], 'shots': 20000}, 0.025: {'shots': 20000, 'max_prob': 0.2122254201645435, 'max_edge': (14, 5), 'ffwd': [0.16313820121434175, 0.163033843927195], 'conjugate': [0.18079999999999996], 'standard': [0.17329800009384494, 0.17086371653538335]}, 0.035: {'shots': 20000, 'max_prob': 0.286846589498413, 'max_edge': (14, 5), 'ffwd': [0.2296182877075076, 0.23057467547772753], 'conjugate': [0.2386], 'standard': [0.24148666862849816, 0.2306988236850487]}, 0.045: {'shots': 20000, 'max_prob': 0.3372661707698559, 'max_edge': (14, 5), 'ffwd': [0.2771476487087339, 0.2789487342809259], 'conjugate': [0.28254999999999997], 'standard': [0.2929482406696155, 0.2868739356846903]}}
results = {'chuqvb9vrup8982kos50': {'shots': 8000, 'max_prob': 0.4226632374629576, 'max_edge': (8, 8), 'ffwd': [0.32809590474255046, 0.14914130540400133], 'conjugate': [0.148875], 'standard': [0.11564024478034608, 0.11459415536228441]}, 'chur07ii3durlgru8vsg': {'shots': 8000, 'max_prob': 0.525584803201192, 'max_edge': (5, 5), 'ffwd': [0.18074559234179782, 0.15291095876019187], 'conjugate': [0.12562500000000001], 'standard': [0.1111984324284314, 0.11090988271688366]}}
